In [1]:
# 0. Basic Libraries importation
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics


In [2]:
#1. Extract and group users data for the year 2020:
df19Dec = pd.read_csv('2019-Dec.csv.gz', compression = 'gzip')
df20Jan = pd.read_csv('2020-Jan.csv.gz', compression = 'gzip')

RawData = pd.concat([df19Dec, df20Jan], 
                    ignore_index = True)


In [3]:
#Delete source csv from memory to free up some memory space for analysis
del df19Dec
del df20Jan

## Data Information
Data is retrieved from: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store and https://rees46.com/.

Data is about users' interactions on a multi-category online shop (similar to online shopping platform such as Shoppee or Lazada). We are using data from 2 months December 2019 and January 2020. This period is at the start of Covid-19 in America where severe effect is yet to be observed.
#### File structure

| Property    | Description |
| :----------- | :----------- |
| event_time  | Time when event happened at (in UTC).   |
| event_type  | Only one kind of event: purchase, view, cart |
| product_id  | ID of a product |
| category_id | Product's category ID |
| category_code | Product's category taxonomy (code name) if it was possible to make it. Usually present for meaningful categories and skipped for different kinds of accessories: cat1.cat2  |
| brand | Downcased string of brand name. Can be missed. |
| price  | Float price of a product. Present. |
| user_id  | Permanent user ID. |
|**user_session**| Temporary user's session ID. Same for each user's session. Is changed every time user come back to online store from a long pause. |


In [5]:
RawData.info(show_counts =True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123509919 entries, 0 to 123509918
Data columns (total 9 columns):
 #   Column         Non-Null Count      Dtype  
---  ------         --------------      -----  
 0   event_time     123509919 non-null  object 
 1   event_type     123509919 non-null  object 
 2   product_id     123509919 non-null  int64  
 3   category_id    123509919 non-null  int64  
 4   category_code  111376181 non-null  object 
 5   brand          108847254 non-null  object 
 6   price          123509919 non-null  float64
 7   user_id        123509919 non-null  int64  
 8   user_session   123509879 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 8.3+ GB


In [6]:
# Since there are missing values in brand and category data,Filter out all actions concerns products without brands or category
RawData.dropna(subset=['brand', 'category_code'],inplace = True)

In [7]:
print(RawData.shape)
RawData.head()

(98927597, 9)


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,view,1005105,2232732093077520756,construction.tools.light,apple,1302.48,556695836,ca5eefc5-11f9-450c-91ed-380285a0bc80
2,2019-12-01 00:00:01 UTC,view,2402273,2232732100769874463,appliances.personal.massager,bosch,313.52,539453785,5ee185a7-0689-4a33-923d-ba0130929a76
4,2019-12-01 00:00:02 UTC,view,20100164,2232732110089618156,apparel.trousers,nika,101.68,517987650,906c6ca8-ff5c-419a-bde9-967ba8e2233e
5,2019-12-01 00:00:02 UTC,view,100008256,2053013561185141473,accessories.umbrella,ikea,163.56,542860793,a1bcb550-1065-4769-a80a-0ccb4bcee78d
7,2019-12-01 00:00:03 UTC,view,1005239,2232732093077520756,construction.tools.light,xiaomi,256.38,525740700,370e8c88-3d07-41df-9aaa-2adf5a0bf312


In [8]:
# Replace event type from string values to an int value to easily sort their values
# For event type, View: 0, Cart: 1 and Purchase: 2.
RawData['event_type'].replace({'view': 0, 'cart': 1, 'purchase': 2}, inplace =True)

In [9]:
# To not lose vital information about the activities in each user_session when we collapse them to the final
# action on each product, we add a new variable called activity_count which includes record all activities a user
# perform in that user session.
RawData['activity_count'] = RawData.groupby('user_session')['user_session'].transform('size')

In [10]:
print(RawData.shape)
RawData.head()

(98927597, 10)


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,activity_count
0,2019-12-01 00:00:00 UTC,0,1005105,2232732093077520756,construction.tools.light,apple,1302.48,556695836,ca5eefc5-11f9-450c-91ed-380285a0bc80,1.0
2,2019-12-01 00:00:01 UTC,0,2402273,2232732100769874463,appliances.personal.massager,bosch,313.52,539453785,5ee185a7-0689-4a33-923d-ba0130929a76,8.0
4,2019-12-01 00:00:02 UTC,0,20100164,2232732110089618156,apparel.trousers,nika,101.68,517987650,906c6ca8-ff5c-419a-bde9-967ba8e2233e,2.0
5,2019-12-01 00:00:02 UTC,0,100008256,2053013561185141473,accessories.umbrella,ikea,163.56,542860793,a1bcb550-1065-4769-a80a-0ccb4bcee78d,1.0
7,2019-12-01 00:00:03 UTC,0,1005239,2232732093077520756,construction.tools.light,xiaomi,256.38,525740700,370e8c88-3d07-41df-9aaa-2adf5a0bf312,3.0


In [12]:
# Interestingly, we can also include a historical variable that tracks the number of time a particular user has access the 
# online shopping platform

# Converting 'event_time' to datetime
RawData['event_time'] = pd.to_datetime(RawData['event_time'])

# Sorting the DataFrame by user_id and event_time
RawData.sort_values(by=['user_id', 'event_time'], inplace=True)

# Grouping by user_id and counting the number of sessions for each user
RawData['user_session_count'] = RawData.groupby('user_id')['user_session'].transform(lambda x: x.factorize()[0])

# Displaying the DataFrame with the appended column
RawData.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,activity_count,user_session_count
105542201,2020-01-21 19:25:37+00:00,0,24900082,2232732111700230936,furniture.kitchen.table,ergolux,3.60,29830839,11a27b53-8fe7-4ee2-be20-332f0987ed6b,1.0,0
45912914,2019-12-22 18:49:44+00:00,0,1004249,2232732093077520756,construction.tools.light,apple,766.53,30493659,4c351de9-7f1f-4626-8ac8-e8ca3b84b751,1.0,0
46440830,2019-12-23 04:50:04+00:00,0,1004249,2232732093077520756,construction.tools.light,apple,766.53,30493659,698942e5-6ab9-42f6-a723-92decae67bba,1.0,1
4390678,2019-12-03 12:34:53+00:00,0,1005098,2232732093077520756,construction.tools.light,samsung,130.75,31198833,e21bbf00-37c6-40ea-8683-24da77b79d12,3.0,0
4394418,2019-12-03 12:37:13+00:00,0,1004838,2232732093077520756,construction.tools.light,oppo,154.42,31198833,e21bbf00-37c6-40ea-8683-24da77b79d12,3.0,0


In [13]:
#Collapsed Data in the same session and product id into one row. 
#Remove all actions by a user on a product, keeping their last action (purchase, cart and view, in that order)
RawData.sort_values(by = "event_type", inplace =True)
RawData.drop_duplicates(subset=['product_id','price', 'user_id','user_session'], keep ='last', inplace =True)
print(RawData.shape)

(62714866, 11)


### Preliminary Analysis of Problem
At this point, we consider the need for data to answer our problem on the characteristics of Online shopping to predict the purchase of a product. It seems like we have a multi-class classification problem of event types: view, cart and purchase. However, since these classes fits into a linear 3 step process (view item, then put item into cart, and then purchase item), instead of doing a one-vs-rest multi-class model (which requires 3 models), we only need to do 2 models, each predicting the probability of a customer will embark on the next step of the online shopping process.

Hence, we decide to split our data set into 2: One contains all data to model for the first step, the other only contains cart and purchase events.

**However, due to time constraint, for this project, we will only consider the second step ie. predict whether a cart item will be purchased or not.**

In [14]:
# Export Cleaned Data as csv file for analysis 
CartPurchaseData = RawData.loc[RawData['event_type'] != 0]
CartPurchaseData.to_csv('CartPurchaseData.csv.gz', index=False, compression='gzip')